# Running a CAT based on a synthetic correct/incorrect 3PL IRT model

This example shows how to run a CAT based on a synthetic correct/incorrect 3PL
IRT model.

Import order is important. We put ComputerAdaptiveTesting last so we get the
extra dependencies.

In [1]:
using Makie
import Pkg
import Random
using Distributions: Normal, cdf
using AlgebraOfGraphics
using ComputerAdaptiveTesting
using ComputerAdaptiveTesting.ExtraDistributions: NormalScaledLogistic
using ComputerAdaptiveTesting.Sim: auto_responder
using ComputerAdaptiveTesting.NextItemRules: AbilityVarianceStateCriterion
using ComputerAdaptiveTesting.TerminationConditions: FixedItemsTerminationCondition
using ComputerAdaptiveTesting.Aggregators: PriorAbilityEstimator, MeanAbilityEstimator, LikelihoodAbilityEstimator
using ComputerAdaptiveTesting.ItemBanks
using ComputerAdaptiveTesting.Integrators
using ComputerAdaptiveTesting.Responses: BooleanResponse
import ComputerAdaptiveTesting.IntegralCoeffs
using CATPlots

@automakie()

Now we are read to generate our synthetic data using the supplied DummyData
module. We generate an item bank with 100 items and fake responses for 3
testees.

In [2]:
using ComputerAdaptiveTesting.DummyData: dummy_full, std_normal, SimpleItemBankSpec, StdModel3PL
using ComputerAdaptiveTesting.MathTraits
(item_bank, question_labels, abilities, responses) = dummy_full(
    Random.default_rng(42),
    SimpleItemBankSpec(StdModel3PL(), OneDimContinuousDomain(), BooleanResponse());
    num_questions=100,
    num_testees=3
)

(ComputerAdaptiveTesting.ItemBanks.GuessItemBank{ComputerAdaptiveTesting.ItemBanks.TransferItemBank{Distributions.Logistic{Float64}}}([0.0, 0.0, 0.0, 0.19462123718477925, 0.03680461950064568, 0.0, 0.0, 0.0, 0.03199394449288699, 0.0  …  0.10170685458856432, 0.0, 0.0, 0.010798570292840827, 0.065242846024754, 0.0, 0.14535402809450157, 0.1270532802604405, 0.0, 0.20752872171066686], ComputerAdaptiveTesting.ItemBanks.TransferItemBank{Distributions.Logistic{Float64}}(Distributions.Logistic{Float64}(μ=0.0, θ=0.5875440658049353), [-0.5226182039316702, 1.6445545468069516, 0.7478715281174344, -0.5880023151678505, 1.3794526558826363, 1.4691874033616144, 2.5021975988724083, -0.8115034987704383, 0.31758883193517146, -0.868874051070674  …  -1.7298587279046034, -0.6326230817592203, -0.35129074883338746, -0.15154473500702556, 0.6413921949231631, -1.5015997471709928, -0.6024810982461476, 1.0199576727767985, 0.11185258594974716, -0.13376943468506888], [1.3054000661136147, 1.294194060695899, 0.90149335370

Simulate a CAT for each testee and record it using CatRecorder.
CatRecorder collects information which can be used to draw different types of plots.

In [3]:
max_questions = 99
integrator = FixedGKIntegrator(-6, 6, 80)
dist_ability_est = PriorAbilityEstimator(std_normal)
ability_estimator = MeanAbilityEstimator(dist_ability_est, integrator)
rules = CatRules(
    ability_estimator,
    AbilityVarianceStateCriterion(dist_ability_est, integrator),
    FixedItemsTerminationCondition(max_questions)
)

points = 500
xs = range(-2.5, 2.5, length=points)
raw_estimator = LikelihoodAbilityEstimator()
recorder = CatRecorder(xs, responses, integrator, raw_estimator, ability_estimator)
for testee_idx in axes(responses, 2)
    tracked_responses, θ = run_cat(
        CatLoopConfig(
            rules=rules,
            get_response=auto_responder(@view responses[:, testee_idx]),
            new_response_callback=(tracked_responses, terminating) -> recorder(tracked_responses, testee_idx, terminating),
        ),
        item_bank
    )
    true_θ = abilities[testee_idx]
    abs_err = abs(θ - true_θ)
end

Make a plot showing how the estimated value evolves during the CAT.
We also plot the 'true' values used to generate the responses.

In [4]:
conv_lines_fig = ability_evolution_lines(recorder; abilities=abilities)
conv_lines_fig

FigureGrid()

Make an interactive plot, showing how the distribution of the ability
likelihood evolves.

In [5]:
conv_dist_fig = lh_evolution_interactive(recorder; abilities=abilities)
conv_dist_fig

Figure()

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*